# This notebook creates a dataset for min/max winning scores
## Prep
- Ensure that database filenames and dates in SPORT_CFG are correct and that the files exist in $FANTASY_HOME
- For LOL make sure that cost csv files in $FANTASY_LINEUP_CACHE_DIR

## Data will include
- min win df score
- max win df score
- median team score (real game scores)
- 75th percentile team score (real game scores)
- number of slate games
- median df score for each player position
- 75th percentile df score for each player position
- median df score of top 50% of players for each position over the previous W weeks
- 75th percentile df score of top 50% for each position over the previous W weeks

In [25]:
# Configuration and imports

from datetime import date
from functools import partial
import re
from typing import Optional, Callable
import os

import pandas as pd

from fantasy_py import ContestStyle
from fantasy_py.lineup.strategy import GeneralPrizePool, FiftyFifty, Contest

from best_possible_lineup_score import get_stat_names


FANTASY_HOME = os.environ['FANTASY_HOME']
CONTEST_DATA_PATH = os.path.join(os.environ['FANTASY_ARCHIVE_BASE'], "betting")

# the datasets to generate, dict mapping sport to dict with keys sport, min_date, max_date, historic data filename
SPORT_CFGS = {
    'mlb': {
        'min_date': date(2019, 1, 1),
        'max_date': date(2021, 1, 1),
        'db_filename': os.path.join(FANTASY_HOME, "mlb_hist_20082021.scored.db"),
        'cost_pos_drop': {'DH', 'RP'},
        'cost_pos_rename': {'SP': 'P'},
    },
    'nfl': {
        'min_date': date(2020, 1, 12),  # no NHL dfs slates before this date
        'max_date': date(2021, 4, 1),
        'db_filename': os.path.join(FANTASY_HOME, "nfl_hist_2009-2020.scored.db"),
    },
    'nba': {
        'min_date': {None: date(2017, 8, 1), 'yahoo': date(2020, 8, 1)},
        'max_date': date(2021, 8, 1),
        'db_filename': os.path.join(FANTASY_HOME, "nba_hist_20082009-20202021.scored.db"),
    },
    'nhl': {
        'min_date': {'draftkings': date(2019, 10, 9),   # dk changed scoring formula for nhl
                     'fanduel': date(2019, 8, 1),       # fd missing positional data prior to 2019 season
                     None: date(2017, 8, 1)},
        'max_date': date(2021, 4, 1),
        'db_filename': os.path.join(FANTASY_HOME, "nhl_hist_20072008-20192020.scored.db"),
        'cost_pos_rename': {'LW': 'W', 'RW': 'W'},
    },
    'lol': {
        'db_filename': os.path.join(FANTASY_HOME, "lol_hist_2014-2021.scored.db"),
        'min_date': date(2020, 1, 1),
        'max_date': date(2021, 1, 1),
        'services': ['draftkings', 'fanduel'],
    }
}


# days to use to identify top players going into a slate
TOP_PLAYER_DAYS = 21
# players above this percentil over the last TOP_PLAYER_DAYS are considered top players
TOP_PLAYER_PERCENTILE = .70


# fanduel/draftkings/yahoo
SERVICES = [
    'draftkings',
    'fanduel',
    'yahoo',
]

STYLES: list[str] = [
    ContestStyle.CLASSIC,
    ContestStyle.SHOWDOWN,
]

CONTEST_TYPES: list[Contest] = [
    FiftyFifty,
    GeneralPrizePool,
]


In [26]:
# get contest data
def infer_contest_style(service, title) -> ContestStyle:
    if service == 'draftkings':
        if ('Showdown' in title or
                re.match('.*.{2,3} vs .{2,3}\)', title)):
            return ContestStyle.SHOWDOWN
        return ContestStyle.CLASSIC
    if service == 'fanduel':
        if '@' in (title or ''):
            return ContestStyle.SHOWDOWN
        return ContestStyle.CLASSIC
    if service == 'yahoo':
        if (' Cup ' in title or
            ' to 1st]' in title or
            ' 50/50' in title or
            'QuickMatch vs ' in title or
            'H2H vs ' in title or
            '-Team' in title or   # N-team contests are classic
            'Freeroll' in title or
            'Quadruple Up' in title or
                'Guaranteed' in title):
            return ContestStyle.CLASSIC
    raise NotImplementedError(
        f"Could not infer contest style for {service=} {title=}")


def infer_contest_type(service, title) -> str:
    if service == 'draftkings':
        if re.match('.* vs\. [^)]+$', title):
            return 'H2H'
        return FiftyFifty.NAME if 'Double Up' in title else GeneralPrizePool.NAME
    if service == 'fanduel':
        if 'Head-to-head' in (title or ''):
            return 'H2H'
        if (title or '').startswith('50/50'):
            return FiftyFifty.NAME
        return GeneralPrizePool.NAME
    if service == 'yahoo':
        if (' QuickMatch vs ' in title or
                'H2H vs ' in title):
            return 'H2H'
        if ' 50/50' in title:
            return FiftyFifty.NAME
        if (' Cup ' in title or
            ' to 1st]' in title or
            'Freeroll' in title or
            'Quadruple Up' in title or
            # multi-team games are GPP if not caught by 50/50
            '-Team' in title or
            # treat winner takes all like a gpp
            title.endswith('Team Winner Takes All') or
                'Guaranteed' in title):
            return GeneralPrizePool.NAME
    raise NotImplementedError(
        f"Could not infer contest type for {service=} {title=}")


def add_bet_links(service, contest_df: pd.DataFrame) -> pd.DataFrame:
    pass


def get_contest_df(service, sport, style, contest_type, min_date, max_date) -> pd.DataFrame:
    """ 
    create a dataframe from the contest dataset
    """
    contest_csv_path = os.path.join(
        CONTEST_DATA_PATH, service + ".contest.csv")
    contest_df = pd.read_csv(contest_csv_path, parse_dates=['date']) \
                   .query('sport == @sport and @min_date <= date < @max_date')[['contest_id', 'date', 'title', 'top_score', 'last_winning_score', 'entries']]
    contest_df.date = contest_df.date.dt.normalize()
    contest_df = contest_df.where(contest_df.notnull(), None)

    # add style and type
    #     with pd.option_context('max_rows', 1000, 'max_colwidth', 100):
    #         display(contest_df)
    contest_df['style'] = contest_df.title.map(
        partial(infer_contest_style, service)
    )
    contest_df['type'] = contest_df.title.map(
        partial(infer_contest_type, service)
    )
    queries = []
    if style is not None:
        # print(f"Filtering for {style=}")
        queries.append('style == @style')
    if contest_type is not None:
        # print(f"Filtering for {contest_type=}")
        queries.append('type == @contest_type.NAME')
    if len(queries) > 0:
        contest_df = contest_df.query(' and '.join(queries))

    betting_csv_path = os.path.join(
        CONTEST_DATA_PATH, 
        service + ".betting.csv"
    )
    bet_df = pd.read_csv(betting_csv_path) \
               .drop_duplicates('contest_id') \
               .set_index('contest_id')[['link']]
    contest_df = contest_df.merge(bet_df, how='left', on='contest_id')
    return contest_df


# contest_df = get_contest_df("draftkings", "lol", ContestStyle.CLASSIC, FiftyFifty, date(2019, 1, 1), date(2020, 1, 1))
# with pd.option_context('max_rows', 1000, 'max_columns', 100, 'max_colwidth', 99999):
#    display(contest_df.sort_values(['style', 'type']))


In [27]:
# player draft dataframe creation

SERVICE_ABBR = {
    'fanduel': 'fd',
    'draftkings': 'dk',
    'yahoo': 'y'
}

# TODO: better solution for LOL would be to take the team name and use team_remaps
# remapping of team abbrs found in draft data to those found in database,
# dict[sport -> dict[service -> dict[draft abbr -> db abbr]]]
# if service is None, then use all services
LOL_ABBR_REMAPS = {
    None: {
        'AF':  'KF',  # Afreeca Freecs -> Kwangdong Freecs
        'AGO': 'RGO',
        'APK': 'SP',
        'EST': 'ES',  # eStar
        'FCS': 'S04',
        'FTC': 'FNC',
        'FQ':  'FLY',
        'IM':  'IMT',
        'ITZ': 'INTZ',
        'MG':  'MSF',
        'ML':  'MAD',
        'OHT': '100',
        'ROG': 'RGE',
        'SB':  'LSB',
        'SK':  'SKG',  # SK Gaming
        'TD':  'NS',
        'VFG': 'GIA',
        'VG':  'RA',
    },
    'fanduel': {
        'ES':  'XL',  # Excel
    },
}


def lol_abbr_remap(service, data_row) -> str:
    """ for lol return a remapped team abbr if there is a remapping"""
    for service_key in [service, None]:
        if service_key in LOL_ABBR_REMAPS and data_row.team_abbr in LOL_ABBR_REMAPS[service_key]:
            return LOL_ABBR_REMAPS[service_key][data_row.team_abbr]
    return data_row.team_abbr


nfl_abbr_remap = lambda service, row : \
    'OAK' if (row.team_abbr == 'LV' and row.date < pd.Timestamp(2020, 7, 1)) else row.team_abbr


ABBR_REMAPPERS: dict[str, Callable[[str, str], str]] = {
    'lol': lol_abbr_remap,
    'nfl': nfl_abbr_remap
}


def get_draft_df(service, sport, style, min_date, max_date) -> pd.DataFrame:
    csv_path = os.path.join(CONTEST_DATA_PATH, service + ".draft.csv")
    draft_df = pd.read_csv(csv_path, parse_dates=['date']) \
                 .query('sport == @sport and @min_date <= date < @max_date')
    assert len(draft_df) > 0, \
        f"no draft data found for {sport=}, {service=}, {style=}, {min_date=}, {max_date=}"

    draft_df['service'] = draft_df.contest.map(
        lambda contest: contest.split('-', 1)[0])
    draft_df.team_abbr = draft_df.team_abbr.str.upper()
    if sport in ABBR_REMAPPERS:
        draft_df.team_abbr = draft_df.apply(
            partial(ABBR_REMAPPERS[sport], service),
            axis=1,
        )
    service_abbr = SERVICE_ABBR[service]
    draft_df = draft_df.query('service == @service_abbr and team_abbr.notnull()')[
        ['position', 'name', 'team_abbr', 'contest_id']]

    return draft_df


# draft_df = get_draft_df(SERVICE, SPORT, STYLE, MIN_DATE, MAX_DATE)
# display(draft_df)


In [28]:
# create team contest dataframes

from fantasy_py import FANTASY_SERVICE_DOMAIN, util


def create_team_contest_df(contest_df, draft_df, service, sport):
    service_cls = util.CLSRegistry.get_class(FANTASY_SERVICE_DOMAIN, service)
    abbr_remaps = service_cls.get_team_abbr_remapping(sport)

    # add team/lineup draft data
    team_contest_df = pd.merge(contest_df, draft_df, on='contest_id')
    team_contest_df.team_abbr = team_contest_df.team_abbr.map(
        lambda abbr: abbr_remaps.get(abbr) or abbr
    )

    return team_contest_df


# team_contest_df = create_team_contest_df(contest_df, draft_df, SERVICE, SPORT)
# print(f"{len(team_contest_df.contest_id.unique())} contests")
# display(team_contest_df)

def common_title(title_series: pd.Series) -> str:
    """ the title of a contest will be the common prefix amongst all the possible contest titles """
    title_list = title_series.tolist()
    if None in title_list:
        return ""
    return os.path.commonprefix(title_list)


def create_teams_contest_df(tc_df):
    """ group contests together and create team sets used in each contest """
    tc_df = pd.DataFrame(
        tc_df.groupby(
            ['contest_id', 'date', 'style', 'type', 'link', 'entries']
        ).agg(
            {'team_abbr': set,
             'title': common_title,
             'top_score': lambda score: score.mean(),
             'last_winning_score': lambda score: score.mean()}
        )
    ).reset_index()
    tc_df = tc_df.rename(columns={'team_abbr': 'teams'})
    tc_df['draft_team_count'] = tc_df.teams.map(len)
    return tc_df


# teams_contest_df = create_teams_contest_df(team_contest_df)
# display(f"{len(teams_contest_df)} team sets")
# display(teams_contest_df)

In [29]:
# load slate data from db
import sqlite3
import pandas as pd


def get_slate_df(db_filename, service, style, min_date, max_date) -> Optional[pd.DataFrame]:
    conn = sqlite3.connect(db_filename)
    sql = f"""
    select distinct daily_fantasy_slate.id as slate_id, date, 
        daily_fantasy_slate.name as slate_name, style as contest_style, abbr
    from daily_fantasy_slate 
        join daily_fantasy_cost on daily_fantasy_slate.id = daily_fantasy_cost.daily_fantasy_slate_id
        join team on team_id = team.id
    where service = '{service}' and date between '{min_date}' and date('{max_date}', '-1 days')
    """

    if style is not None:
        sql += f" and style = '{style.name}'"

    # print(sql)
    db_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
    # with pd.option_context('max_rows', 100):
    #     display(db_df)
    conn.close()
    if len(db_df) == 0:
        return None

    # get team sets
    slate_db_df = pd.DataFrame(
        db_df.groupby(
            ['slate_id', 'date', 'slate_name', 'contest_style']
        ).agg(
            {'abbr': set}
        )
    ).reset_index()

    try:
        slate_db_df = slate_db_df.set_index('date') \
                                 .rename(columns={'abbr': 'teams'})
    except Exception as ex:
        raise ValueError("Error processing slate db df", slate_db_df) from ex

    slate_db_df['team_count'] = slate_db_df.teams.map(len)
    return slate_db_df


# slate_db_df = get_slate_df('/fantasy/lol_hist_2014-2021.scored.db', "draftkings", ContestStyle.CLASSIC, '2020-09-09', '2020-09-10')
# with pd.option_context('display.max_rows', 100):
#     display(slate_db_df)


In [30]:
# get slate id
import numpy as np
from typing import Optional

NO_SLATE_ID_FOUND = pd.Series({'slate_id': None, 'team_count': None})

def get_slate_id(contest_row, slate_db_df) -> pd.Series:
    """ 
    guesses the db slate id contest_row
    returns - series of (slate_id, number of teams playing in slate)
    """
    try:
        date_slates = slate_db_df.loc[[
            contest_row.date]].sort_values('team_count')
    except KeyError as ke:
        print(f"get_slate_id:: Key error/No slates found for {contest_row.date}")
        return NO_SLATE_ID_FOUND
    try:
        slates = date_slates.query("@contest_row.teams <= teams")
    except Exception as e:
        print(f"get_slate_id:: Unhandled exception querying for teams date {contest_row.date}: {e}")
        return NO_SLATE_ID_FOUND

    slates_found = len(slates)
    if slates_found == 0:
        with pd.option_context('display.max_colwidth', None):
            display(f"No slates found for {contest_row.date} that matches teams {contest_row.teams}. date_slates_df=",
                    date_slates)
        return NO_SLATE_ID_FOUND

    return slates.iloc[0][['slate_id', 'team_count']]

# slate_ids_df = teams_contest_df.apply(get_slate_id, axis=1)
# display(slate_ids_df)


In [31]:
# slate game score info

def create_team_score_df(db_filename, slate_ids_str, top_player_percentile) -> Optional[pd.DataFrame]:
    conn = sqlite3.connect(db_filename)
    sql = f"""
    select distinct daily_fantasy_slate.id as slate_id, game.id as game_id, 
           game.score_home, game.score_away
    from daily_fantasy_slate
        join daily_fantasy_cost on daily_fantasy_slate.id = daily_fantasy_cost.daily_fantasy_slate_id
        join game on ((game.date = daily_fantasy_slate.date or 
		               game.dt between daily_fantasy_slate.date and datetime(daily_fantasy_slate.date, '+1 days', '+6 hours')) and
                      game.season = daily_fantasy_slate.season and 
                      (daily_fantasy_cost.team_id in (game.away_team_id, game.home_team_id)))
    where daily_fantasy_slate.id in ({slate_ids_str})
    """

    # print("team score sql\n", sql)
    db_team_score_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
    conn.close()
    # display("team score df", db_team_score_df)
    if len(db_team_score_df) == 0:
        return None

    team_score_df = db_team_score_df.melt(id_vars=['slate_id', 'game_id'], value_vars=['score_home', 'score_away']) \
        .groupby(['slate_id']) \
        .agg({'value': ['median', lambda x: np.percentile(x, top_player_percentile * 100)]})
    team_score_df.columns = ['team-med',
                             f'team-{top_player_percentile * 100}th_pctl']
    return team_score_df


# for mlb double headers this will cause inaccuracy for players that played in both games
# slate_ids_str = ','.join(map(str, slate_ids_df.slate_id.dropna()))
# team_score_df = create_team_score_df(DB_FILENAME, slate_ids_str, TOP_PLAYER_PERCENTILE)
# display(team_score_df)


In [32]:
# get position scores

def get_exploded_pos_df(db_filename, sport, service_abbr, slate_ids_str,
                        cost_pos_drop: Optional[set], cost_pos_rename: Optional[dict]) -> Optional[pd.DataFrame]:
    conn = sqlite3.connect(db_filename)
    stat_names = get_stat_names(sport, service_abbr, as_str=True)

    # for mlb double headers this query will cause inaccuracy for players that played in both games
    # games have a date equal to the slate date or must have a datetime starting prior to 6am on the following date
    sql = f"""
    select daily_fantasy_slate.id as slate_id, positions as cost_positions, 
        player_position.abbr as stat_position, 
        value as score, daily_fantasy_cost.team_id, daily_fantasy_cost.player_id
    from daily_fantasy_slate
        join daily_fantasy_cost on 
           daily_fantasy_slate.id = daily_fantasy_cost.daily_fantasy_slate_id
        join game on (
           (game.date = daily_fantasy_slate.date or 
		    game.dt between daily_fantasy_slate.date and datetime(daily_fantasy_slate.date, '+1 days', '+6 hours')) and
           game.season = daily_fantasy_slate.season and 
           (daily_fantasy_cost.team_id in (game.away_team_id, game.home_team_id))
        )
        join calculation_datum on (
            calculation_datum.game_id = game.id and 
            calculation_datum.player_id is daily_fantasy_cost.player_id and
            calculation_datum.team_id = daily_fantasy_cost.team_id
        )
        join statistic on calculation_datum.statistic_id = statistic.id
        join player on daily_fantasy_cost.player_id = player.id
        join player_position on player.player_position_id = player_position.id
    where daily_fantasy_slate.id in ({slate_ids_str}) and
        statistic.name in ({stat_names})
    """
    # print("Exploded POS data:\n", sql)

    db_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
    conn.close()

    if len(db_df) == 0:
        return None

    # TODO: only need to test for 'Unknown' so long as it is still stored in DB as a cost position value
    def apply_func(row): 
        return row.stat_position if row.cost_positions is None else row.cost_positions
    db_df['position'] = db_df.apply(
        apply_func,
        axis=1
    )

    db_exploded_pos_df = db_df.assign(position=db_df.position.str.split('/')) \
                              .explode('position')

    if cost_pos_drop is not None:
        db_exploded_pos_df = db_exploded_pos_df.query(
            'position not in @cost_pos_drop')
    if cost_pos_rename is not None:
        for old_pos, new_pos in cost_pos_rename.items():
            db_exploded_pos_df.loc[db_exploded_pos_df.position ==
                                   old_pos, 'position'] = new_pos
    return db_exploded_pos_df


def get_position_scores(db_exploded_pos_df, top_player_percentile):
    db_pos_scores_df = db_exploded_pos_df[['slate_id', 'position', 'score']] \
        .groupby(['slate_id', 'position']) \
        .agg(['median', lambda x: np.percentile(x, top_player_percentile * 100)])
    db_pos_scores_df.columns = ['med-dfs',
                                f'{top_player_percentile * 100}th-pctl-dfs']
    db_pos_scores_df = db_pos_scores_df.reset_index(level='position') \
        .pivot(columns='position', values=['med-dfs', f'{top_player_percentile * 100}th-pctl-dfs'])
    return db_pos_scores_df


# SPORT = 'lol'
# SERVICE = 'draftkings'

# db_exploded_pos_df = get_exploded_pos_df(DB_FILENAME, SPORT, SERVICE_ABBR[SERVICE], slate_ids_str)
# display(db_exploded_pos_df)
# db_pos_scores_df = get_position_scores(db_exploded_pos_df, TOP_PLAYER_PERCENTILE)
# display(db_pos_scores_df)


In [33]:
def get_player_scores(db_filename, db_exploded_pos_df,
                      sport, service_abbr, top_player_days, min_date, max_date):
    """ Get top player scores (e.g. players that are likely to be highly drafted) """
    conn = sqlite3.connect(db_filename)
    stat_names = get_stat_names(sport, service_abbr, as_str=True)

    sql = f"""
    select game.date, calculation_datum.player_id, calculation_datum.team_id, calculation_datum.value as score 
    from game
        join calculation_datum on calculation_datum.game_id = game.id
        join statistic on calculation_datum.statistic_id = statistic.id
    where statistic.name in ({stat_names}) 
        and date between date('{min_date}', '-{top_player_days} days') and date('{max_date}', '-1 days')
    """
    # print(sql)
    db_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
    conn.close()
    # display(db_df)

    db_filtered_df = db_df.query(
        '(player_id in @db_exploded_pos_df.player_id) '
        'or (player_id.isnull() and team_id in @db_exploded_pos_df.team_id)'
    )
    return db_filtered_df


# db_filtered_df = get_player_scores(DB_FILENAME, SPORT, SERVICE_ABBR[SERVICE], TOP_PLAYER_DAYS, MIN_DATE, MAX_DATE)
# display(db_filtered_df)


In [34]:
def create_predict_df(teams_contest_df, slate_ids_df, team_score_df, db_pos_scores_df, top_lineup_scores) -> pd.DataFrame:
    """
    join contest, slate id, team score and player position scores
    """

    dfs = [
        teams_contest_df[['date', 'style', 'type',
                          'top_score', 'last_winning_score', 'link']],
        top_lineup_scores,
        slate_ids_df,
    ]
    predict_df = pd.concat(dfs, axis='columns') \
                   .join(team_score_df, on='slate_id') \
                   .join(db_pos_scores_df, on='slate_id')
    return predict_df


# predict_df = create_predict_df(teams_contest_df, slate_ids_df, team_score_df, db_pos_scores_df)
# with pd.option_context('max_columns', 100):
#     display(predict_df)


# filename = f"{SPORT}-{SERVICE}-{STYLE.name}-{CONTEST_TYPE.NAME}.csv"
# print(f"Writing data to file '{filename}'")
# predict_df.to_csv(filename, index=False)


In [35]:
# generate dataset

from best_possible_lineup_score import (
    TopScoreCacheMode, best_possible_lineup_score, best_score_cache
)


def generate_dataset(
    sport, cfg, service, style, contest_type,
    min_date=None, max_date=None, max_count: Optional[int] = None,
    top_score_cache_mode: TopScoreCacheMode = 'default',
    datapath: str = "data",
) -> pd.DataFrame:
    """
    max_count - maximum number of slates to process
    min_date - includsive
    max_date - not inclusive
    top_score_cache_mode - 
        'default'=load and use the cache, 
        'overwrite'=overwrite all existing cache data if any exists
        'missing'=use all existing valid cache data, any cached failures will be rerun
    """
    assert (min_date is None) or (max_date is None) or min_date < max_date, \
        "invalidate date range. max_date must be greater than min_date. Or one must be None"
    filename = f"{sport}-{service}-{style.name}-{contest_type.NAME}.csv"
    # print(f"Creating data for file '{filename}'")

    db_filename = cfg['db_filename']
    if min_date is None:
        min_date = cfg['min_date']
    if max_date is None:
        max_date = cfg['max_date']

    contest_df = get_contest_df(
        service, sport, style, contest_type, min_date, max_date)
    if contest_df is not None:
        contest_df = contest_df.head(max_count)

    draft_df = get_draft_df(service, sport, style, min_date, max_date)
    # display(draft_df)

    team_contest_df = create_team_contest_df(
        contest_df, draft_df, service, sport)
    # display(f"{len(team_contest_df.contest_id.unique())} contests", team_contest_df)

    teams_contest_df = create_teams_contest_df(team_contest_df)
    # with pd.option_context('display.max_rows', 100, 'display.max_colwidth', None):
    #     display(f"{len(teams_contest_df)} slate team sets in team_contest_df",
    #              teams_contest_df)

    slate_db_df = get_slate_df(db_filename, service, style, min_date, max_date)
    if slate_db_df is None:
        raise ValueError("No slates found for", service,
                         style, min_date, max_date)
    # with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    #     display("Slate db df", slate_db_df)

    slate_ids_df = teams_contest_df.apply(
        get_slate_id, axis=1, args=(slate_db_df, ))
    # display(slate_ids_df)

    if len(slate_ids_df) == 0:
        raise ValueError("No slates ids found (based on teams contest df)")

    try:
        # need this for subsequent sql queries
        slate_ids_str = ','.join(
            map(str, slate_ids_df.slate_id.dropna().astype(int)))
    except Exception as ex:
        raise ValueError("Something wrong with slate_ids_df",
                         slate_ids_df) from ex

    if len(slate_ids_str) == 0:
        raise ValueError("No slate ids found after removing Nones")
    team_score_df = create_team_score_df(
        db_filename, slate_ids_str, TOP_PLAYER_PERCENTILE)
    if team_score_df is None:
        raise ValueError("Empty team score df")
    # display("team score df", team_score_df)

    db_exploded_pos_df = get_exploded_pos_df(
        db_filename, sport, SERVICE_ABBR[service], slate_ids_str,
        cfg.get('cost_pos_drop'), cfg.get('cost_pos_rename'),
    )

    if db_exploded_pos_df is None:
        raise ValueError("No exploded positional data returned!")
    # display(db_exploded_pos_df)

    db_pos_scores_df = get_position_scores(
        db_exploded_pos_df, TOP_PLAYER_PERCENTILE)
    # display(db_pos_scores_df)

    # db_filtered_df = get_player_scores(
    #     db_filename, db_exploded_pos_df,
    #     sport, SERVICE_ABBR[service], TOP_PLAYER_DAYS, min_date, max_date
    # )
    # display(db_filtered_df)

    # cache for top scores
    with best_score_cache(sport, top_score_cache_mode, cache_dir=datapath) as top_score_dict:
        best_possible_lineup_score_part = partial(best_possible_lineup_score,
                                                  db_filename,
                                                  SERVICE_ABBR[service],
                                                  sport=sport,
                                                  best_score_cache=top_score_dict)

        top_lineup_scores = slate_ids_df.slate_id.map(
            best_possible_lineup_score_part
        )

    top_lineup_scores.name = 'best-possible-score'
    predict_df = create_predict_df(
        teams_contest_df, slate_ids_df, team_score_df, db_pos_scores_df, top_lineup_scores)
    # with pd.option_context('max_columns', 100):
    #     display("predict df", predict_df)

    filepath = os.path.join(datapath, f"{sport}-{service}-{style.name}-{contest_type.NAME}.csv")
    print(f"Writing data to '{filepath}'")
    predict_df.to_csv(filepath, index=False)
    return predict_df


In [36]:
TOP_SCORE_CACHE_MODE: TopScoreCacheMode = 'default'
SPORTS = ['nfl'] # SPORT_CFGS:
for sport in SPORTS:
    cfg_min_date = SPORT_CFGS[sport]['min_date']
    cfg_max_date = SPORT_CFGS[sport]['max_date']
    for service in SPORT_CFGS[sport].get('services', SERVICES):
        min_date = cfg_min_date.get(service, cfg_min_date.get(None)) \
            if isinstance(cfg_min_date, dict) else cfg_min_date
        max_date = cfg_max_date.get(service, cfg_max_date.get(None)) \
            if isinstance(cfg_max_date, dict) else cfg_max_date
        for style in STYLES:
            for contest_type in CONTEST_TYPES:
                print(
                    f"Processing {sport}, {service}, {style}, {contest_type.NAME}")
                try:
                    df = generate_dataset(sport, SPORT_CFGS[sport], service, style, contest_type,
                                          min_date=min_date, max_date=max_date,
                                          top_score_cache_mode=TOP_SCORE_CACHE_MODE)
                    with pd.option_context('display.max_rows', 1000, 'display.max_columns', 100):
                        display(df)
                except ValueError as ex:
                    failure = ex
                    print(
                        f"********************* Error for {sport}, {service}, {style}, {contest_type.NAME}: {ex}")

print("Done!")


  0%|          | 0/3 [00:00<?, ?it/s]

Processing nfl, draftkings, classic, FIFTY_FIFTY


"No slates found for 2020-11-15 00:00:00 that matches teams {'NYG', 'PIT', 'LV', 'MIA', 'PHI', 'NO', 'LAC', 'ARI', 'CAR', 'HOU', 'DET', 'GB', 'SEA', 'WAS', 'LA', 'DEN', 'TB', 'BUF'}. date_slates_df="

,slate_id,slate_name,contest_style,teams,team_count
date,,,,,
2020-11-15,388,(Early Only),CLASSIC,"{NYG, PHI, CAR, HOU, DET, CLE, GB, WAS, JAC, TB}",10
2020-11-15,385,Unnamed-CLASSIC-Slate-41097,CLASSIC,"{NYG, DET, JAC, BUF, MIA, NO, WAS, SEA, LA, DEN, LV, PHI, SF, HOU, CLE, TB, LAC, ARI, CAR, GB}",20
2020-11-15,389,(Sun-Mon),CLASSIC,"{NYG, DET, CHI, JAC, BUF, MIA, NO, WAS, SEA, LA, MIN, DEN, LV, PHI, SF, HOU, CLE, TB, LAC, ARI, NE, CAR, GB, BAL}",24
2020-11-15,387,Unnamed-CLASSIC-Slate-41498,CLASSIC,"{NYG, DET, CHI, JAC, BUF, MIA, NO, WAS, SEA, LA, MIN, DEN, LV, PHI, SF, HOU, CLE, IND, TB, LAC, TEN, ARI, NE, CAR, GB, BAL}",26


Exiting best_score_cache


...

Writing data to 'data/nfl-yahoo-CLASSIC-GPP.csv'


,date,style,type,top_score,last_winning_score,link,best-possible-score,slate_id,team_count,team-med,team-70.0th_pctl,"(med-dfs, QB)","(med-dfs, RB)","(med-dfs, TE)","(med-dfs, WR)","(70.0th-pctl-dfs, QB)","(70.0th-pctl-dfs, RB)","(70.0th-pctl-dfs, TE)","(70.0th-pctl-dfs, WR)"
0,2020-01-19,classic,GPP,190.16,133.76,https://sports.yahoo.com/dailyfantasy/contest/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-09-13,classic,GPP,206.10,140.60,https://sports.yahoo.com/dailyfantasy/contest/...,245.46,519.0,26.0,23.5,27.0,17.18,3.70,1.10,3.20,20.820,8.02,2.96,7.12
2,2020-09-13,classic,GPP,209.06,NaN,https://sports.yahoo.com/dailyfantasy/contest/...,245.46,519.0,26.0,23.5,27.0,17.18,3.70,1.10,3.20,20.820,8.02,2.96,7.12
3,2020-09-20,classic,GPP,215.88,NaN,https://sports.yahoo.com/dailyfantasy/contest/...,255.60,524.0,28.0,28.0,31.0,16.43,3.00,0.90,4.80,26.262,8.33,4.50,8.40
4,2020-09-27,classic,GPP,226.50,NaN,https://sports.yahoo.com/dailyfantasy/contest/...,271.60,529.0,28.0,28.0,31.0,16.07,2.95,1.75,3.80,20.800,7.45,5.49,8.00
5,2020-10-04,classic,GPP,219.30,NaN,https://sports.yahoo.com/dailyfantasy/contest/...,266.66,534.0,28.0,27.0,31.0,19.90,3.30,2.90,4.60,23.776,8.56,5.38,7.30
6,2020-10-04,classic,GPP,200.38,128.10,https://sports.yahoo.com/dailyfantasy/contest/...,266.66,534.0,28.0,27.0,31.0,19.90,3.30,2.90,4.60,23.776,8.56,5.38,7.30
7,2020-10-11,classic,GPP,200.20,NaN,https://sports.yahoo.com/dailyfantasy/contest/...,248.10,539.0,26.0,28.0,32.0,14.22,3.00,1.05,4.10,23.134,7.20,3.15,8.74
8,2020-10-11,classic,GPP,194.46,138.86,https://sports.yahoo.com/dailyfantasy/contest/...,248.10,539.0,26.0,28.0,32.0,14.22,3.00,1.05,4.10,23.134,7.20,3.15,8.74
9,2020-10-18,classic,GPP,221.00,NaN,https://sports.yahoo.com/dailyfantasy/contest/...,260.70,544.0,24.0,23.5,30.1,15.40,3.05,1.50,2.10,19.250,6.95,4.95,7.66


Processing nfl, yahoo, showdown, FIFTY_FIFTY
********************* Error for nfl, yahoo, showdown, FIFTY_FIFTY: ('No slates found for', 'yahoo', <ContestStyle.SHOWDOWN: 'showdown'>, datetime.date(2020, 1, 12), datetime.date(2021, 4, 1))
Processing nfl, yahoo, showdown, GPP
********************* Error for nfl, yahoo, showdown, GPP: ('No slates found for', 'yahoo', <ContestStyle.SHOWDOWN: 'showdown'>, datetime.date(2020, 1, 12), datetime.date(2021, 4, 1))



100%|██████████| 1/1 [00:04<00:00,  4.91s/it]


Done!
